In [2]:
#필요한 모듈 임포트하기
import pandas as pd
from pandas import DataFrame  as df
import numpy as np
from collections import Counter
from konlpy.tag import Twitter

%matplotlib inline
import matplotlib.pyplot as plt
import re 
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
from wordcloud import WordCloud
from datetime import datetime

In [3]:
twitter = Twitter()


C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
def data_cleansing_character(d):                      # 정규표현식을 활용 but, 목차 제거 등은 다른 방법이 필요한 듯
    parse = re.sub('[-=+,#/\?:^$.@*\"※~⚫&%ㆍ!』\n\"\'\\‘|\(\)\[\]\<\>`\'…》]'," ",d)          #parse = re.sub('[^\w\s]',"",d)   이것도 되나?
    return parse

In [5]:
def data_ckonlpy(twitter, d):
    konlpy_array = twitter.morphs(d)
    return konlpy_array

In [6]:
def data_cleansing(twitter, name):
    
    data_word = []
    
    data_cleansing_file = open(name,'r',-1,'utf-8')           # -1 은 버퍼 / utf-8 추가 or 파일 인코딩 ANSI 변경으로 인코딩 문제 해결 가능
    data = data_cleansing_file.read()

    c = data_cleansing_character(data)
    
    return data_ckonlpy(twitter, c)

In [7]:
#형태소 분석 결과 파일 저장
def prepare_data_set(data_list, data_set):
    cnt = 0
    for word in data_list:
        if len(word) <= 1:            # 단어 길이가 1 이면 --> 무의미하다고 가정
            continue
        if word == " ":  # 단어 길이가 1 이면 --> 무의미하다고 가정
            continue
        data_set.append(word)
        cnt += 1

In [8]:
for i in range(1,5):
    name = str(i)+'.txt'

    data_set = []
    
    data_word = data_cleansing(twitter, name)
    
    prepare_data_set(data_word, data_set)
    
    

In [9]:
def all_data_set(twitter, data):
    
    for i in range(1,5):
        name = str(i)+'.txt'

        data_set = []

        data_word = data_cleansing(twitter, name)

        prepare_data_set(data_word, data_set)

        
        data.append(data_set)
        
       

In [10]:
data = []

all_data_set(twitter, data)

In [11]:
print(data[0])

['LG', 'SK', '법원', '배터리', '자료', '내야', '이번', '기술', '유출', '놓고', '대립', '전기차', '배터리', '영업비밀', '유출', '여부', '놓고', '송전', '돌입', 'LG', '화학', '051910', 'SK', '이노베이션', '096770', '이번', '에는', '소송', '으로', '인한', '기술', '유출', '가능성', '으로', '대립', '하고', '있다', 'SK', '이노베이션', '배터리', '핵심', '기술', '담긴', '자료', '로펌', '미국', '법원', '제출', '되어', '경쟁', '으로', '유출', '있다', '주장', 'LG', '화학', '외부', '유출', '가능성', '전혀', '없다', '맞서고', '있다', '회사', '소송', '위해', '관련', '기술', '자료', '해외', '보내려면', '정부', '승인', '받아야', '하는데', '정부', '이를', '승인', '해줄', '지도', '주목', '된다', 'LG', '화학', '지난달', '29일', '미국', '국제무역', '위원회', 'ITC', '델라웨어', '지방법원', 'SK', '이노베이션', '2017년', '부터', '2년', '76', '명의', '핵심', '인력', '빼가는', '영업', '비밀', '침해', '했다', '소송', '냈다', 'SK', '이노베이션', '공개', '채용', '통해', '자발', '으로', '지원', '경력', '후보자', '에서', '채용', '해왔다', '반박', '했다', '전기차', '배터리', '소송', '위해', 'LG', '화학', '세계', '최대', '규모', '로펌', '덴톤스', '선임', '했다', 'SK', '이노베이션', '미국', '대형', '로펌', '코빙턴', '앤드', '벌링', '법률', '대리인', '으로', '선임', '맞서고', '있다', '배터리', '기술', '입증', '위해', '해외', '보내려면', '정

In [14]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=data, size=20, window=5, min_count=3, workers=4, sg=0)

WARNING - under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay


In [15]:
model_result = model.wv.most_similar("LG")
print(model_result)

[('유출', 0.6562955379486084), ('기술', 0.64632248878479), ('산업', 0.6210224032402039), ('한국', 0.5877012610435486), ('판단', 0.5806751847267151), ('기업', 0.5691313147544861), ('국가', 0.5186683535575867), ('에도', 0.5132547616958618), ('보호', 0.48019078373908997), ('리튬', 0.4761490523815155)]


In [16]:
model_result = model.wv.most_similar("SK")
print(model_result)

[('이다', 0.6453231573104858), ('미국', 0.6155555248260498), ('배터리', 0.5945542454719543), ('기술', 0.521084189414978), ('영업비밀', 0.515106737613678), ('통해', 0.5122043490409851), ('제조', 0.5068944096565247), ('에서', 0.49555715918540955), ('기업', 0.48936226963996887), ('승인', 0.48328498005867004)]


In [17]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [19]:
model_result = loaded_model.most_similar("미국")
print(model_result)

[('SK', 0.6155555248260498), ('외부', 0.6024956107139587), ('정보', 0.59092116355896), ('에서는', 0.5889157652854919), ('이다', 0.5597110390663147), ('해외', 0.5486270189285278), ('법인', 0.5344054102897644), ('에서', 0.5300325751304626), ('배터리', 0.5293235182762146), ('했다', 0.5066293478012085)]


In [26]:
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(data[0:3])]
tagged_data

[TaggedDocument(words=['LG', 'SK', '법원', '배터리', '자료', '내야', '이번', '기술', '유출', '놓고', '대립', '전기차', '배터리', '영업비밀', '유출', '여부', '놓고', '송전', '돌입', 'LG', '화학', '051910', 'SK', '이노베이션', '096770', '이번', '에는', '소송', '으로', '인한', '기술', '유출', '가능성', '으로', '대립', '하고', '있다', 'SK', '이노베이션', '배터리', '핵심', '기술', '담긴', '자료', '로펌', '미국', '법원', '제출', '되어', '경쟁', '으로', '유출', '있다', '주장', 'LG', '화학', '외부', '유출', '가능성', '전혀', '없다', '맞서고', '있다', '회사', '소송', '위해', '관련', '기술', '자료', '해외', '보내려면', '정부', '승인', '받아야', '하는데', '정부', '이를', '승인', '해줄', '지도', '주목', '된다', 'LG', '화학', '지난달', '29일', '미국', '국제무역', '위원회', 'ITC', '델라웨어', '지방법원', 'SK', '이노베이션', '2017년', '부터', '2년', '76', '명의', '핵심', '인력', '빼가는', '영업', '비밀', '침해', '했다', '소송', '냈다', 'SK', '이노베이션', '공개', '채용', '통해', '자발', '으로', '지원', '경력', '후보자', '에서', '채용', '해왔다', '반박', '했다', '전기차', '배터리', '소송', '위해', 'LG', '화학', '세계', '최대', '규모', '로펌', '덴톤스', '선임', '했다', 'SK', '이노베이션', '미국', '대형', '로펌', '코빙턴', '앤드', '벌링', '법률', '대리인', '으로', '선임', '맞서고', '있다', '배터리', '기술', '입증', 

In [27]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [28]:
## Train doc2vec model
model = Doc2Vec(tagged_data, vector_size=20, window=1, min_count=1, workers=4, epochs = 100)
# Save trained doc2vec model
model.save("test_doc2vec.model")
## Load saved doc2vec model
model= Doc2Vec.load("test_doc2vec.model")
## Print model vocabulary
model.wv.vocab

{'LG': <gensim.models.keyedvectors.Vocab at 0x1b86be8dec8>,
 'SK': <gensim.models.keyedvectors.Vocab at 0x1b86be8d248>,
 '법원': <gensim.models.keyedvectors.Vocab at 0x1b86be8d2c8>,
 '배터리': <gensim.models.keyedvectors.Vocab at 0x1b86be8d348>,
 '자료': <gensim.models.keyedvectors.Vocab at 0x1b86be8d408>,
 '내야': <gensim.models.keyedvectors.Vocab at 0x1b86be8d4c8>,
 '이번': <gensim.models.keyedvectors.Vocab at 0x1b86be8d508>,
 '기술': <gensim.models.keyedvectors.Vocab at 0x1b86be8d548>,
 '유출': <gensim.models.keyedvectors.Vocab at 0x1b86be8d5c8>,
 '놓고': <gensim.models.keyedvectors.Vocab at 0x1b86be8d648>,
 '대립': <gensim.models.keyedvectors.Vocab at 0x1b86be8d6c8>,
 '전기차': <gensim.models.keyedvectors.Vocab at 0x1b86be8d748>,
 '영업비밀': <gensim.models.keyedvectors.Vocab at 0x1b86be8d808>,
 '여부': <gensim.models.keyedvectors.Vocab at 0x1b86be8d8c8>,
 '송전': <gensim.models.keyedvectors.Vocab at 0x1b86be8d908>,
 '돌입': <gensim.models.keyedvectors.Vocab at 0x1b86be8d948>,
 '화학': <gensim.models.keyedvectors.V

In [29]:
model.docvecs.most_similar(positive=[model.infer_vector(data[3])],topn=5)

[(0, 0.964752197265625), (1, 0.4249908924102783), (2, 0.30961936712265015)]

In [34]:
from glove import Glove

In [35]:
corpus = Corpus() 
corpus.fit(result, window=5)
# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성

glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.

NameError: name 'Corpus' is not defined